# Test 9

Various GS from Simulated Annealing

Field: 8mT along the x axis

In [72]:
import os
import sys

import numpy as np
import pandas as pd
import random
from math import isclose

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Ignore some messages from pandas

from tqdm import tqdm
import matplotlib.pyplot as plt

sys.path.insert(0, '../icenumerics/')
import icenumerics as ice
from auxiliary import *
from montecarlo_tools import *
from IPython.display import clear_output

ureg = ice.ureg

%reload_ext autoreload
%autoreload 2

idx = pd.IndexSlice

In [58]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'

In [59]:
# Definition of colloids

sp = ice.spins()

trapSep = 10*ureg.um
particleRadius = 5*ureg.um
L = 30*ureg.um
N = 10

# Create simulation objects
sp.create_lattice("square",[N,N],lattice_constant=L, border="periodic")

particle = ice.particle(radius = particleRadius,
            susceptibility = 0.0576,
            diffusion = 0.125*ureg.um**2/ureg.s,
            temperature = 300*ureg.K,
            density = 1000*ureg.kg/ureg.m**3)

trap = ice.trap(trap_sep = trapSep,
            height = 4*ureg.pN*ureg.nm,
            stiffness = 1e-3*ureg.pN/ureg.nm)

col = ice.colloidal_ice(sp, particle, trap,
            height_spread = 0, 
            susceptibility_spread = 0.1,
            periodic = True)


col.randomize()
col.region = np.array([[0,0,-3*(particleRadius/L/N).magnitude],[1,1,3*(particleRadius/L/N).magnitude]])*N*L

# Simulated Annealing

In [60]:
def simulation(colloid, field, dataPath, niter = 100):

    # Initialize stuff
    energies = np.zeros(niter)

    # Save initial state
    colloid.to_ctrj().to_csv(dataPath + "trj0.csv")

    # No sé por qué debo de calcular esto primero xd
    Eold = colloid.calculate_energy(B = field).magnitude
    energies[0] = Eold

    T = 300

    for k in tqdm(range(1,niter)):
        
        newcolloid = colloid.copy(deep = True)
        newcolloid = flip_colloids(newcolloid,amount=3)
        newcolloid = flip_loop(newcolloid,a=30,size=10)

        Enew = newcolloid.calculate_energy(B=field).magnitude
        dE = Enew - Eold

        # Accept or reject the change
        if is_accepted(dE,T):
            colloid = newcolloid.copy(deep = True)
            #colloid = newcolloid
            Eold = Enew
            energies[k] = Eold
        else:
            energies[k] = Eold
        
        
        currentFile = f"trj{k}.csv"
        colloid.to_ctrj().to_csv(dataPath + currentFile)
        
        T = 0.9*T
    
    return colloid, energies


In [ ]:
angle = (57) * (np.pi/180)
B0 = 8*ureg.mT
field = [np.sin(angle),0,np.cos(angle)]*B0
dataPath = "../data/test9/57/"

np.random.seed(0)
final_state, data = simulation(col, field, dataPath, niter=10000)

pd.DataFrame(data).to_csv(dataPath + "data.csv")

## Plotting energy

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(data)
ax.set_ylabel("Energy")
ax.set_xlabel("MC Steps")

In [ ]:
data[-1] - final_state.calculate_energy(B=field).magnitude

## Plotting states

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
v = ice.vertices()
v = v.colloids_to_vertices(col)
col.display(ax)
v.display()
fig.savefig("initial.png")

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
v = ice.vertices()
v = v.colloids_to_vertices(final_state)
final_state.display(ax)
v.display()
fig.savefig("final.png")

# Plotting all states

In [73]:
def load_saved_state(main_path,folder):
    file = "/trj9999.csv"
    return pd.read_csv(main_path+folder+file,index_col=0)

def load_saved_energy(main_path,folder):
    file = "/data.csv"
    return pd.read_csv(main_path+folder+file,index_col=0)["0"].to_list()

In [74]:
main_path = "../data/test9/mc/"
angles = os.listdir(main_path)

final_states = { directory:get_colloids_from_ctrj(load_saved_state(main_path,directory),particle,trap) for directory in tqdm(angles) }
all_energies = { directory:load_saved_energy(main_path,directory) for directory in angles }


100%|██████████| 25/25 [00:06<00:00,  3.94it/s]


In [75]:
fig_path = "../data/test9/figures/"
for cur_angle in angles:
    clear_output(wait=True)

    print(f"Doing... {cur_angle} degrees")

    cur_state = final_states[cur_angle]
    cur_energy = all_energies[cur_angle]

    string = "$\\theta$ = "+cur_angle+"$^\circ$"

    fig, axes = plt.subplots(1,2,figsize=(12,5))
    ax = axes[0]
    v = ice.vertices()
    v = v.colloids_to_vertices(cur_state)
    cur_state.display(ax)
    v.display(ax)

    ax = axes[1]
    ax.plot(all_energies[cur_angle])
    ax.set_xlabel("MC Steps")
    ax.set_ylabel("Energy \, [pN$\cdot$nm]")
    plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0), useMathText=True)

    fig.text(0.5,0.9,string,ha='center',fontsize=20)

    file = cur_angle+".png"
    fig.savefig(fig_path+file)

    plt.close()

Doing... 75 degrees


100%|██████████| 200/200 [00:01<00:00, 138.67it/s]
